# Query 3

In [1]:
from sedona.spark import *
from pyspark.sql.functions import col, sum, collect_list, regexp_replace
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, count, avg
import time

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2026,application_1732639283265_1986,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def measure_join_performance(df, description):
    start_time = time.time()
    df.count()  # Trigger the join
    end_time = time.time()
    print(f"{description} took {end_time - start_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Query 3") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)

# Load the crime data, LA income and RE codes csv 
crime_data_1 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True,)
crime_data_2 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True,)
la_income = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv", header=True, inferSchema=True)
police_stations = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv", header=True, inferSchema=True)
explanation = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks_fields.csv", header=True, inferSchema=True)
crime_data = crime_data_1.union(crime_data_2)


geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")

# Formatting magic
flattened_blocks_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
comm_pop.filter(col("COMM") == ("Little Tokyo")).select("COMM", "ZCTA10", "total_population", "total_households").show(100, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------+----------------+----------------+
|COMM        |ZCTA10|total_population|total_households|
+------------+------+----------------+----------------+
|Little Tokyo|90013 |1956            |1184            |
|Little Tokyo|90012 |1179            |893             |
+------------+------+----------------+----------------+

In [6]:
# Filter out unpopulated regions or regions without a name
populated_blocks = flattened_blocks_df.filter((flattened_blocks_df.POP_2010 != 0) & (flattened_blocks_df.HOUSING10 != 0) & (flattened_blocks_df.COMM != " "))

# Calculate total population per region-Zip Code pair (e.g., "Community-Zip Code", "COMM-ZCTA10")
comm_pop = populated_blocks.groupBy("COMM", "ZCTA10").agg(
    sum("POP_2010").alias("total_population"),
    sum("HOUSING10").alias("total_households"),
)

# Clean the 'Estimated Median Income' column from $ characters
la_income_clean = la_income.withColumn(
    "Estimated Median Income",
    regexp_replace(col("Estimated Median Income"), "[^0-9]", "").cast("double")
)

# Join the dataframe containing the total_population and total_housing with the income
comm_income = comm_pop.join(
    la_income_clean, 
    ((la_income_clean["Community"].contains(comm_pop["region"])) 
      & (comm_pop["zip_code"] == la_income_clean["Zip Code"])), "inner"
).drop(la_income_clean["Community"])


# Use the "explain" function to find out the optimizer's choice on join strategy
comm_income.explain(True)

# Experiment with join strategies 
comm_income_broadcast = comm_pop.join(
    la_income_clean.hint("broadcast"),
    ((la_income_clean["Community"].contains(comm_pop["COMM"])) 
      & (comm_pop["ZCTA10"] == la_income_clean["Zip Code"])), "inner"
)

comm_income_merge = comm_pop.join(
    la_income_clean.hint("merge"),
   ((la_income_clean["Community"].contains(comm_pop["COMM"])) 
      & (comm_pop["ZCTA10"] == la_income_clean["Zip Code"])), "inner"
)


comm_income_shuffle_hash = comm_pop.join(
    la_income_clean.hint("shuffle_hash"),
  ((la_income_clean["Community"].contains(comm_pop["COMM"])) 
      & (comm_pop["ZCTA10"] == la_income_clean["Zip Code"])), "inner"
)

comm_income_shuffle_replicate_nl = comm_pop.join(
    la_income_clean.hint("shuffle_replicate_nl"),
   ((la_income_clean["Community"].contains(comm_pop["COMM"])) 
      & (comm_pop["ZCTA10"] == la_income_clean["Zip Code"])), "inner"
)

# Measure their performances 
measure_join_performance(comm_income_broadcast, "Broadcast Join")
measure_join_performance(comm_income_merge, "Sort-Merge Join")
measure_join_performance(comm_income_shuffle_hash, "Shuffle Hash Join")
measure_join_performance(comm_income_shuffle_replicate_nl, "Replicated Nested Loop Join")

# Keep only the columns of interest in the comm_income
comm_income_final = comm_income.select(
    "region", 
    "Zip Code",
    "total_population",
    col("Estimated Median Income").alias("median_income"),
    "total_households"
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Crimes Per Person

In [21]:
crime_pop.filter(col("region").contains("Little Tokyo")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------+------------+--------+----------------+----------------+------------------+
|        COMM|ZCTA10|      region|zip_code|total_population|total_households|total_crime_number|
+------------+------+------------+--------+----------------+----------------+------------------+
|Little Tokyo| 90013|Little Tokyo|   90013|            1956|            1184|              9405|
|Little Tokyo| 90012|Little Tokyo|   90012|            1179|             893|              4209|
+------------+------+------------+--------+----------------+----------------+------------------+

In [12]:
region_totals.filter(col("region").contains("Little Tokyo")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------------------+-------------------------+
|      region|region_total_population|region_total_crime_number|
+------------+-----------------------+-------------------------+
|Little Tokyo|                   3135|                    13614|
+------------+-----------------------+-------------------------+

In [22]:
comm_income_final.filter(col("region").contains("Little Tokyo")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------+----------------+-------------+----------------+
|      region|Zip Code|total_population|median_income|total_households|
+------------+--------+----------------+-------------+----------------+
|Little Tokyo|   90012|            1179|      31576.0|             893|
+------------+--------+----------------+-------------+----------------+

In [25]:
# Create geometry column for crime data
crime_data = crime_data.filter((col("LON") != 0) | (col("LAT") != 0)) 
crime_data_geom = crime_data.withColumn("geom", ST_Point(col("LON"), col("LAT")))
crime_data_location = crime_data_geom.join(populated_blocks, ST_Within(crime_data_geom["geom"], flattened_blocks_df["geometry"]), "inner")
#crime_data_location.explain(True)

# Group the crime_pop dataframe by "COMM" column, and count the number of crimes commited per region.
crime_num = crime_data_location.groupBy("COMM", "ZCTA10").agg(
    count("*").alias("total_crime_number")
)

# Join income and population data with crime data
crime_pop = comm_pop.join(
    crime_num,
    on= ["COMM", "ZCTA10"]
)
'''
crime_income.explain(True)

# Experiment with join strategies 
crime_income_broadcast = zip_income_weight.join(
    crime_pop.hint("broadcast"),
    (comm_income["region"] == crime_pop["COMM"]) 
)

crime_income_shuffle_hash = zip_income_weight.join(
    crime_pop.hint("shuffle_hash"),
   (comm_income["region"] == crime_pop["COMM"]) 
)

crime_income_merge = zip_income_weight.join(
    crime_pop.hint("merge"),
    (comm_income["region"] == crime_pop["COMM"]) 
)

crime_income_shuffle_replicate_nl = zip_income_weight.join(
    crime_pop.hint("shuffle_replicate_nl"),
    (comm_income["region"] == crime_pop["COMM"]) 
)

# Measure Performances
measure_join_performance(crime_income_broadcast, "Broadcast Join")
measure_join_performance(crime_income_merge, "Sort-Merge Join")
measure_join_performance(crime_income_shuffle_hash, "Shuffle Hash Join")
measure_join_performance(crime_income_shuffle_replicate_nl, "Replicated Nested Loop Join")
'''

# Calculate total region population
region_totals = crime_pop.groupBy("region").agg(
    sum("total_population").alias("region_total_population_crime"),
    sum("total_crime_number").alias("region_total_crime_number")
)


income_totals = comm_income_final.groupBy("region").agg(
    sum("total_population").alias("region_total_population_income")
)

# Join the total population of each region and calculate the income per person and crime per person


income_result_join = comm_income_final.join(income_totals,  on="region")
income_result = income_result_join.groupBy("region").agg(
    sum(col("total_households") * col("median_income")/col("region_total_population_income")).alias("income_per_person"),
)

crime_result = region_totals.groupBy("region").agg(
    avg(col("region_total_crime_number")/col("region_total_population_crime")).alias("crimes_per_person")
)

result = income_result.join(crime_result, on="region")
    
'''
# Experiment with join strategies 
result_broadcast = zip_income_weight.join(
    crime_pop.hint("broadcast"),
    (comm_income["region"] == crime_pop["COMM"]) 
)

result_shuffle_hash = zip_income_weight.join(
    crime_pop.hint("shuffle_hash"),
   (comm_income["region"] == crime_pop["COMM"]) 
)

result_merge = zip_income_weight.join(
    crime_pop.hint("merge"),
    (comm_income["region"] == crime_pop["COMM"]) 
)

result_shuffle_replicate_nl = zip_income_weight.join(
    crime_pop.hint("shuffle_replicate_nl"),
    (comm_income["region"] == crime_pop["COMM"]) 
)

# Measure Performances
measure_join_performance(result_broadcast, "Broadcast Join")
measure_join_performance(result_merge, "Sort-Merge Join")
measure_join_performance(result_shuffle_hash, "Shuffle Hash Join")
measure_join_performance(result_shuffle_replicate_nl, "Replicated Nested Loop Join")
'''


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\n# Experiment with join strategies \nresult_broadcast = zip_income_weight.join(\n    crime_pop.hint("broadcast"),\n    (comm_income["region"] == crime_pop["COMM"]) \n)\n\nresult_shuffle_hash = zip_income_weight.join(\n    crime_pop.hint("shuffle_hash"),\n   (comm_income["region"] == crime_pop["COMM"]) \n)\n\nresult_merge = zip_income_weight.join(\n    crime_pop.hint("merge"),\n    (comm_income["region"] == crime_pop["COMM"]) \n)\n\nresult_shuffle_replicate_nl = zip_income_weight.join(\n    crime_pop.hint("shuffle_replicate_nl"),\n    (comm_income["region"] == crime_pop["COMM"]) \n)\n\n# Measure Performances\nmeasure_join_performance(result_broadcast, "Broadcast Join")\nmeasure_join_performance(result_merge, "Sort-Merge Join")\nmeasure_join_performance(result_shuffle_hash, "Shuffle Hash Join")\nmeasure_join_performance(result_shuffle_replicate_nl, "Replicated Nested Loop Join")\n'

In [27]:
result.orderBy(col("crimes_per_person").desc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------+------------------+
|       region| income_per_person| crimes_per_person|
+-------------+------------------+------------------+
|       Vernon| 4406.446428571428| 6.339285714285714|
|     Downtown| 20983.81025571273| 4.343322386425835|
| Little Tokyo| 23916.34266327396| 4.342583732057416|
|    Hollywood|25729.159548647393|1.5561056370854967|
|    Chinatown| 14476.74654501543|1.2941097544612907|
|  Rancho Park|39037.730430606694|1.2649271650392189|
|       Venice| 45830.39010055233|1.2587081501521813|
|  Westchester|31762.894136060662| 1.222845399666053|
| Leimert Park| 17064.34163701068|1.2153918664152976|
|Baldwin Hills|16900.587688153224|1.1941330539549502|
|    Hyde Park|14104.141176470588|1.0831036827601017|
| Park La Brea| 36647.89450437442|1.0259916758685128|
|     Westlake|10641.940877225634|1.0030754585728408|
|        Watts| 7755.219963684051|0.9951830929083022|
|  Studio City| 45258.06898840832|0.9419970631424376|
|      Melrose| 38250.009557